# Streamlit web application

In [ ]:
! pip install streamlit pyngrok python-dotenv

In [ ]:
import os
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata

In [ ]:
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_token)

In [ ]:
def run_streamlit():
  os.system('streamlit run /content/app.py --server.port 8501')

In [ ]:
%%writefile app.py
import cv2
import numpy as np
import streamlit as st
import tensorflow as tf
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing import image
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import google.generativeai as genai
from google.colab import userdata
import PIL.Image
import os
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

output_dir = 'saliency_maps'
os.makedirs(output_dir, exist_ok=True)

def generate_explanations(img_path, model_prediction, confidence):
  prompt = f"""You are an expert neurologist. You are tasked with explaining of explaining a saliency map of a brain MRI scan.
  The saliency map was generated by a deep learning model that was trained to classify brain tumors as either glicoma meningioma, pituitary, or no tumor.
  The saliency map highlights the regions of the image that the machine learning model is focusing on to make the prediction.
  The deep learning model predicted the image to be of class '{model_prediction}' with a confidence of {confidence * 100}%
  In your response:
  -Explain what regions of the brain the model is focusing on, based on the saliency map. Refer to the regions highlighted in light cyan
  those are the regions that the model is focusing on.
  - Explain why the model made this prediction
  - Don't mention anything like 'the saliency map highlights the regions of the brain the model is focusing on, which are light cyan' in your response.
  - Keep your reponse the 4 sentences max.

  Let's think about this step by step. Verify step by step.
  """
  img = PIL.Image.fromarray(img_path)

  model = genai.GenerativeModel(model_name="gemini-1.5-flash")
  response = model.generate_content([prompt, img])

  return response.text

def generate_seliency_map(model, img_array, class_index, img_size):
  with tf.GradientTape() as tape:
    img_tensor = tf.convert_to_tensor(img_array)
    tape.watch(img_tensor)
    predictions = model(img_tensor)
    target_class = predictions[:, class_index]
  gradients = tape.gradient(target_class, img_tensor)
  gradients = tf.math.abs(gradients)
  gradients = tf.reduce_max(gradients, axis=-1)
  gradients = gradients.numpy().squeeze()

  # Resize gradients to match original image size
  gradients = cv2.resize(gradients, img_size)

  center = (gradients.shape[0] // 2 , gradients.shape[1] // 2)
  radius = min(center[0], center[1]) - 10
  y, x = np.ogrid[:gradients.shape[0], :gradients.shape[1]]
  mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2

  # Apply mask to gradients
  gradients = gradients * mask

  # Normalize only the brain area
  brain_gradients = gradients[mask]

  if brain_gradients.max() > brain_gradients.min():
    brain_gradients = (brain_gradients - brain_gradients.min()) / (brain_gradients.max() - brain_gradients.min())
  gradients[mask] = brain_gradients

  # Apply a higher treshold
  threshold = np.percentile(gradients[mask], 80)
  gradients[gradients < threshold] = 0

  # Apply a more aggresive soomthing
  gradients = cv2.GaussianBlur(gradients, (11, 11), 0)

  # Create a heatmap overlay with enhanced contrast
  heatmap = cv2.applyColorMap(np.uint8(255 * gradients), cv2.COLORMAP_JET)
  heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

  # Resize the heatmap
  heatmap = cv2.resize(heatmap, img_size)

  # Superimpose the heatmap on the original image with increased opacity
  original_img = image.array_to_img(img)
  original_img = np.array(original_img)
  superimpose_img = heatmap * 0.7 + original_img * 0.3
  superimpose_img = superimpose_img.astype(np.uint8)

  img_path = os.path.join(output_dir, uploaded_file.name)
  with open(img_path, 'wb') as f:
    f.write(uploaded_file.getbuffer())
  # Save the saliency map
    saliency_map_path = os.path.join(output_dir, f"saliency_{uploaded_file.name}")
    cv2.imwrite(saliency_map_path, cv2.cvtColor(superimpose_img, cv2.COLOR_RGB2BGR))

  return superimpose_img

def load_xception_model(model_path):
    image_shape = (299, 299, 3)
    base_model = tf.keras.applications.Xception(
        weights='imagenet',
        input_shape=image_shape,
        include_top=False,
        pooling='max'
    )

    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.25),
        Dense(4, activation='softmax')
    ])
    model.build((None, )+ image_shape)
    # Compile the model
    model.compile(
        optimizer=Adamax(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )

    # Load weights
    model.load_weights(model_path)
    print(model.summary())  # To verify the architecture and layer shapes
    return model
st.title("Brain Tumor Classification")
st.write("Upload a Brain MRI Scan to classify")
uploaded_file = st.file_uploader("Choose a brain MRI ...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:

  select_model = st.radio("Select model", ("Transfer Learning - Xception", "CNN"))
  if select_model == "Transfer Learning - Xception":
    model = load_xception_model('/content/xception_model.weights.h5')
    image_size = (299, 299)
  else:
    model = load_model('/content/cnn_model.keras')
    image_size = (224, 224)

  labels = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
  img = image.load_img(uploaded_file, target_size=image_size)
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0

  prediction = model.predict(img_array)
  class_index = np.argmax(prediction[0])
  result = labels[class_index]

  saliency_map_path = generate_seliency_map(model, img_array, class_index, image_size)

  col1, col2 = st.columns(2)
  with col1:
    st.image(img, caption='Uploaded Image', use_container_width=True)
  with col2:
    st.image(saliency_map_path, caption='Saliency Map', use_container_width=True)

  st.write("## Classification Results")
  result_container = st.container()
  result_container = st.container()
  result_container.markdown(
      f"""
      <div style="background-color: #000000; color: #ffffff; padding: 30px; border-radius: 15px;">
        <div style="display: flex; justify-content: space-between; align-items: center;">
          <div style="flex: 1; text-align: center;">
            <h3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Prediction</h3>
            <p style="color: #ff0000; font-size: 36px; font-weight: 800; margin: 0;">
              {result}
            </p>
          </div>
          <div style="width: 2px; height: 80px; background-colort: #ffffff; margin: 0 20px;"></div>
          <div style="flex: 1; text-align: center;">
            <H3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Confidence</h3>
            <p style="color: #ff0000; font-size: 36px; font-weight: 800; margin: 0;">
              {prediction[0][class_index] * 100:.2f}%
            </p>
          </div>
        </div>
      </div>
      """,
      unsafe_allow_html=True
  )

  # Prepare data for Plotly chart
  probabilities = prediction[0]
  sorted_indices = np.argsort(probabilities)[::-1]
  sorted_labels = [labels[i] for i in sorted_indices]
  sorted_probabilities = probabilities[sorted_indices]

  # Create a Plotly bar chart
  fig = go.Figure(go.Bar(
      x=sorted_probabilities,
      y=sorted_labels,
      orientation='h',
      marker_color=['red' if label == result else 'blue' for label in sorted_labels]
  ))

  # Customize the chart layout
  fig.update_layout(
      title='Probabilities for each class',
      xaxis_title='Probability',
      yaxis_title='Class',
      height=400,
      width=600,
      yaxis=dict(autorange='reversed')
  )

  # Add value labels to the bars
  for i, prob in enumerate(sorted_probabilities):
      fig.add_annotation(
          x=prob,
          y=i,
          text=f'{prob:.4f}',
          showarrow=False,
          xanchor='left',
          xshift=5
      )

  # Display the Plotly chart
  st.plotly_chart(fig)

  explanation = generate_explanations(saliency_map_path, result, prediction[0][class_index])
  st.write("## Explanation")
  st.write(explanation)


Overwriting app.py


In [ ]:
thread = Thread(target=run_streamlit)
thread.start()

In [ ]:
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print("Public Url: ",public_url)

Public Url:  NgrokTunnel: "https://86e9-35-237-13-208.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
  print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
  ngrok.disconnect(tunnel.public_url)

Overwriting .env
